Django提高02
====
**Request和response对象**

web框架最基础、最核心的任务是：建立请求request和响应response之间的处理机制，即对一个请求，怎样做出响应，并反馈怎样的状态信息。

在Django系统中，当请求页面时：
- Django创建一个HttpRequest对象，其中包含有关请求的元数据。
- URLConf的 path 中定义请求调用的视图。
- Django加载URLConf中定义的视图，将HttpRequest作为第一个参数传递给view函数。
- 每个视图都负责返回一个HttpResponse对象。

在Django的 django.http 模块中有两类对象分别用于处理 Request 和 response：

- HttpRequest objects
- HttpResponse objects

HttpRequest对象由Django自动创建，而创建HttpResponse则是设计者的任务。

编写的每个视图都负责填充参数，并返回一个HttpResponse实例化对象。

# HttpRequest

`class HttpRequest`

当Django接受到一个请求时，就会自动创建一个HttpRequest实例化对象，其中包含由请求的元数据。

我们主要是使用这个HttpRequest对象的属性和方法。

## HttpRequest通过app代码设置的属性

在django配置中不设置 HttpRequest 的 `current_app` 和 `urlconf` 两个属性，但是可以通过自己创建的 app 来设置。

**`HttpRequest.current_app`**

url模板标记将使用该属性的值作为 reverse() 的 current_app 参数

**`HttpRequest.urlconf`**

在代码中设置该属性值，将用作当前请求的根 URLconf ，覆盖 ROOT_URLCONF 设置。请参阅 Django 如何处理详细请求。

urlconf可以设置为 None 以恢复先前中间件所做的任何更改并返回使用 ROOT_URLCONF。

# HttpResponse

`class HttpResponse`

HttpResponse 是一个构造函数。接受参数，构造并返回一个 HttpResonse对象。

创建实例化HttpResponse对象web设计者的主要任务之一。编写的每个视图都负责实例化，填充和返回一个实例化的 HttpResponse 对象。

**用法：**

HttpResponse 用法主要有四个方面：

传递字符串；
传递迭代器；
设置头部字段；
告诉浏览器将响应视为文件附件。

## 传递字符串

HttpRespons 构造典型用法是将页面内容作为字符串传递给HttpResponse构造函数：

典型用法是将页面内容作为字符串传递给HttpResponse构造函数：

```
>>> from django.http import HttpResponse
>>> response = HttpResponse("Here's the text of the Web page.")
>>> response = HttpResponse("Text only, please.", content_type="text/plain")
```

如果要以增量方式添加内容，可以将响应当作类似文件的对象（可以使用它的write()方法）：
```
>>> response = HttpResponse()
>>> response.write("<p>Here's the text of the Web page.</p>")
>>> response.write("<p>Here's another paragraph.</p>")
```

## 传递迭代器

现在还可以传递一个迭代器给 HttpResponse，而不是字符串。HttpResponse将立即使用迭代器，将其内容存储为字符串，然后丢弃它。 具有 close() 方法的对象（如文件和生成器）会立即关闭。

如果需要将响应从迭代器流式传输到客户端(而不是迭代后生成的字符串)，则必须使用 StreamingHttpResponse 类。

## 设置头部字段

每个Response都有一个header部分。参见[HTTP协议详解](Tips-HTTP协议详解.ipynb)

要设置或删除Response中的 header 部分中的字段，将其当作字典处理：

```
>>> response = HttpResponse()
>>> response['Age'] = 120
>>> del response['Age']
```

**请注意**，与字典不同，如果 header 字段不存在，del不会引发KeyError。

为了设置 Cache-Control 和 Vary 的 header 字段，建议使用django.utils.cache中的patch_cache_control() 和 patch_vary_headers() 方法，因为这些字段可以有多个以逗号分隔的值。 “patch” 方法确保其他值，例如 由中间件添加，不会被删除。

HTTP header 字段不能包含换行符。 尝试设置包含换行符（CR或LF）的标题字段将引发BadHeaderError。

## 告诉浏览器将响应视为文件的附件

要告诉浏览器将响应视为文件附件，使用content_type参数并设置Content-Disposition的头部。 例如，下面这段代码可以返回Microsoft Excel电子表格的方式：

```
>>> response = HttpResponse(my_data, content_type='application/vnd.ms-excel')
>>> response['Content-Disposition'] = 'attachment; filename="foo.xls"'
```

没有关于Content-Disposition 标头的Django特定内容。